# CeBrA and SPS Analysis

This notebook will outline a general outline of how you can gain match, combine, and energy calibrate the CeBrA detectors as well as energy calibrate and combine data from the light-ion focal plane detector. This guide uses several different coding languages and programs that one does not necessarily need to use. If you find a better way, then great! But this is how I did it, and I'm sure it can be improved upon and made more efficient. 

## Programs Used

1. SPS_CEBRA_EventBuilder

    This C++ program can be found here: https://github.com/alconley/SPS_CEBRA_EventBuilder. It converts CoMPASS data from experiment at SPS (and CeBrA) to ROOT, sorts the data in time, builds events, and performs preliminary analysis by providing basic histograms.

2. hdtv

    This python and C++ program can be found here: https://github.com/janmayer/hdtv. It allows you to enter in spectrum to then further analyze the data. 

3. Python

    My analysis is mainly through python as that is the coding language I am most comfortable in. You are more than welcome to do the equivalent of this analysis in a different coding language.


## SPS Focal Plane Energy Calibration and Combining Multiple Run Sets

When you first measure data, it is outputted in terms of focal plane position, not energy. So this procedure will guide you through the process of converting the spectra from focal plane position to energy. This stage requires proper cuts on the data for the specific reaction that you want to study. This is done through the EventBuilder and ROOT TBrowser.

1. Open hdtv in the terminal and load in the ROOT file containing the histograms into hdtv (in the histograms directory) by using the following commands: 'root open /path/to/file' followed by 'root get xavg_bothplanes_Cut'.

2. Choose several peaks to fit gaussians to on hdtv. Press 'r' to set the region, 'b' to set regions for background subtraction, and 'p' where you think the peak is. 'Shift + F' then does the fit. You can save these fits by typing 'fit store' and after fitting a bunch of peaks, type 'fit list' to see all the parameters for the peaks you fit. 

3. Now you have to assign these peaks to energies you think they are. Use nndc and other resources to help guide you, including papers that have the same spectrum. Open an excel sheet or libreoffice spreadsheet and make columns with the focal plane position and the energies you think they are. 

4. Fit a second-order polynomial to this data and try to make it as linear as possible (within reason).

5. Repeat 1-4 for each set of data (same magnetic field settings).

Once you have done the previous steps to a satisfactory level (make sure to cross reference and check your calibrations), we now need to combine the different set of runs. Most of the time, you change the magnetic field settings to make a "sweep" across the spectrum with some overlap regions. It may seem easy to just combine everything, but we cannot simply do that because the statistics are not normalized to anything yet. For example, one run set may have an artificial large amount of counts simply because it has more runs (longer runtime). In other words, the cross section of a state should remain the same no matter what the magnetic field is.

There are two ways we can normalize these runs:

1. Pick a few peaks from a run set and normalize every other run set to it.

2. Normalize with respect to the beam current integrator (BCI) of each run. 

### 1. Normalizing with respect to a specific run set

You can look at the notebook 'xavgnormalization.ipynb' to see how I approached this problem with my prior data set. I tried my best to comment what I did.

1. First, we want to check for focal plane losses (particles that hit x1 but not x2 so they do not factor into the xavg spectrum). The focal plane detector tends to have more losses towards the edges and it is good practice to check how significant that is. Open the x1_oneplane and x1_bothplanes spectra, fit some peaks, and take the ratio of the volume of the peaks. This ratio can then be multiplied to the respective peak in the xavg_bothplanes spectrum. 

2. Once that is done, we will proceed like the steps before. Pick a run set and open the xavg spectrum into hdtv. 

3. Find which peaks overlap with other run(s), fit a guassian to that peak, and record the volume. 

4. Take the ratios of the peaks from both run sets. You can decide to take an average of a few peaks, but also keep in mind the losses that you see and make a determination of which peaks are more reliable.

5. Multiply the counts of the other run set by this ratio. One way to do this is to create an array of the same length as the run set and use the 'weights' option within plt.hist or np.histogram with that array. Example: np.full(len(xavgcal113_134), 1.14).

6. With this new weighted run set, repeat the procedure for the other runs that do not overlap with the standard run set. 

7. Check your work by performing residual plots as seen in 'xavgnormalization.ipynb'.

8. Once you are convinced that the normalization is okay, then combine the data into one big histogram!


### 2. Normalizing with respect to the Beam Current Integrator

A more comprehensive outline of cross sections is given in the pdf 'Differential Cross-Section Calculations and Angular Distributions at the SE-SPS' written by Bryan Kelly, but I will give a brief outline here. 

The differential cross section is given by,

\begin{equation}
\frac{d\sigma}{d\Omega} = \frac{N_p}{N_b \cdot F_{target} \cdot \Delta \Omega }
\end{equation}

where $N_p$ is the number of outgoing particles from the scattering chamber, $N_b$ is the number of incident beam particles, $F_{target}$ is the target thickness, and $\Delta\Omega$ is the outgoing beam area. In this procedure, we are going to focus on $N_b$.

\begin{equation}
    N_b = \frac{Q_b}{Z_b \cdot e}

\end{equation}

where $Q_b$ is the total charge of all beam particles incident to the target, $Z_b$ is the proton number of your beam, and $e$ is the elementary unit of charge. To obtain the total charge of all incident beam particles, we use scalar values that are inherent to our experimental setup and are to be recorded during the experiment. The BCI allows us to monitor the current of our beam down the beam-line in our facility. The two scalar values we need to determine the total charge of incident particles is the scale of BCI, which may vary from run to run, and the total number of events the BCI sees. The scale of the BCI is to be recorded during each run, and the BCI counts are read in to a digitizing board during our runs and can be found in the data files generated. Therefore, to get the total charge of particles incident,

\begin{equation}
    Q_b = \frac{\beta \cdot BCIscale}{100Hz}

\end{equation}

where $\beta$ is the total number of BCI events read, and the 100 Hz factor is the sampling rate of the BCI itself. Now if we look at the first equation again we can see the ratio of $N_p$ and $N_b$ are equal to a constant ($F_{target}$ and $\Delta \Omega$ do not change with different runs):

\begin{equation}
\frac{N_p}{N_b} = \frac{d\sigma}{d\Omega} \cdot F_{target} \cdot \Delta \Omega = Constant.
\end{equation}

$N_p$ for an excited state is simply measured by fitting a guassian curve to the respective peak. So what we need to do to normalize by the BCI is calculate $N_b$ and divide that from our integrated peaks. The procedure is shown below:


1. First, we need to find where the Beam Current Integrator is. Go to a run's raw_data folder.

2. Go to the UNFILTERED folder.

3. Open the Hcompass ROOT file in ROOT (TBrowser).

4. Open CH5 histogram (it should be a sharp peak, not the CeBrA CH5)

5. Entries in legend = $\beta$.

6. Record $\beta$ and the BCIscale for each run in a run set in a spreadsheet.

7. Calculate $Q_b$ for each run.

8. Calculate $N_b$ for each run.

9. Combine $N_b$ from each run in a run set to find the total $N_b$. 

10. 1/$N_b$ will be your ratio you will use to normalize peaks, similar to the ratio in the other normalization procedure. 

11. Multiply the counts of the run set by this ratio. One way to do this is to create an array of the same length as the run set and use the 'weights' option within plt.hist or np.histogram with that array. Example: np.full(len(xavgcal113_134), 1.14).

12. Repeat this procedure for each run set.

13. Check your work by creating residual plots to make sure the peak volumes agree between different run sets.

## Gain Matching CeBrA and Combining Coincident Data

Before we can combine and energy calibrate the CeBrA detectors, we must first gain-match their spectra. Because of the different voltage settings and different sizes (2x2 vs 3x4), their peaks may not align very well. So the first objective is to set a standard and shift all the other runs to that one standard. For you standard, choose a spectrum that has many counts and clear, sharp peaks. 

1. First, we will get all the cebra spectra and save them to their own root files, so that they're easier to work with. The following code takes the analyzed runs created by the eventbuilder (analyzed folder in working directory), takes the cebra data, creates histograms, and saves each histogram to its own root file. Each file corresponds to one run and is named accordingly. This is saved as the file 'histstocalibrate.py'.

In [ ]:
import ROOT
import os

# Use this code as the first step in the cebra gain-matching analysis.
# It goes through each analyzed root file in your working directory and gives out just the cebraE{i} columns.
# Each new root file will have a folder for each detector. You can then hadd them in terminal into one root file.
# Use this root file in hdtv to fit and save peaks for gain matching. 

# Function that creates 1D histograms of the cebra spectra
def histo1d(Frame, Name, X, X_List):
   #X_List should be something like X_List = [bins,initial,final]
   histo1d = Frame.Histo1D((f"{Name}",f"{Name}",X_List[0], X_List[1], X_List[2]),X)
   return histo1d


path = '/home/dhoulihan/Projects/SPS_CEBRA_Oct2022ad/Workingdir/analyzed/' # Put in your own analyzed directory

#Creates a list of run numbers give it the path.  Directory should only have the trees that you want to use
run_list = []
for filename in os.listdir(path):
   if filename.endswith(".root"):
       run_number = int(filename.split("_")[1].split(".")[0])
       run_list.append(run_number)

run_list.sort()


# this for loop goes through and creates root files for each run. Each file contains the cebra spectra for the specific run.
for i in range(len(run_list)):
   
   print(f"Run {run_list[i]}")
   
   df = ROOT.RDataFrame("SPSTree",f"{path}run_{run_list[i]}.root")
   cebraE_raw_to_ECal = []
   for j in range(5):     
       df_i = df.Filter(f"cebraE{j} != -1")
       cebraE_raw_to_ECal.append(histo1d(df_i,f"cebraE{j}_run_{run_list[i]}",f"cebraE{j}",[512,0,4096]))

   output = ROOT.TFile.Open(f"run_{run_list[i]}_cebraEnergy.root", "RECREATE")
   output.cd()
   for k in range(5):
       output.mkdir(f"det{k}")
       output.cd(f"det{k}")
       cebraE_raw_to_ECal[k].Write()
   
   output.Close()

2. Once we have the files made, we can then use the command "hadd" on the terminal. This will combine all the spectra into one root file with folders containing all the spectra for a specific detector. Use the following command in the directory with all the root files: hadd  new_root_file.root  run*.root.

3. In hdtv, open the new root file containing the spectra using "root open 'filepath'" and then "root get 'det0/*'" with the number 0 referring to the detector number.

4. Choose several peaks to fit gaussians to on hdtv. Shift + Q auto fits peaks, Shift + F stores the fits, and using Page up or Page Down will change the spectrum. You may need to change the range for the auto fit region, you can do this with the following command: config set  fit.quickfit.region 10, with 10 being the size of the region. Change that number to what you feel is best. 

5. It is usually good to go through using Page Up/Down and take notes of runs that have very low counts such that it would be hard to fit peaks. I would omit those by putting them in a new folder in the 'analyzed' directory and redo the procedure starting from step 1.

6. Go through each spectrum and fit the same peaks for all the runs in one detector folder.

7. Once you have all the fits for one detector, use the following code to write each set of fits to a separate xml file. This way, you can import them into python for future analysis. 

In [ ]:
# path of the analyzed root files, make sure only the good ones are here
# this is to get the run numbers that will be used later
path = '/home/dhoulihan/Projects/SPS_CEBRA_Oct2022ad/Workingdir/analyzed/' 

#Creates a list of run numbers give it the path.  Directory should only have the trees that you want to use
run_list = []
for filename in os.listdir(path):
   if filename.endswith(".root"):
       run_number = int(filename.split("_")[1].split(".")[0])
       run_list.append(run_number)

run_list.sort()

detnum = 0 # detector number 
fitspath = 'fits/det_0/' # directory where you want the output file to go
output_file = f"fits_write_det{detnum}.txt" # name of output file
    
with open(output_file, 'w') as file: # Open the file in write mode

    for i in range(len(run_list)):
        # Write the "s s i" hdtv command
        file.write(f"s s {i}\n")

        # Write the command for writing the fits
        file.write(f"fit write {fitspath}/cebraE{detnum}_run_{run_list[i]} \n")

8. Execute this file in hdtv using the following command: execute 'filename.txt', assuming the file is in the same directory. It will then execute the commands on each line, producing a .fit file for each run in that detector.

9. Repeat steps 6-8 for each detector.

10. One useful thing to do is to create a text file containing hdtv commands that would read in the fit parameters that you exported using the following code (it's basically the same code, just slightly altered). 

In [ ]:
detnum = 0 # detector number 
fitspath = 'fits/det_0/' # directory where you want the output file to go
output_file = f"fits_read_det{detnum}.txt" # name of output file
    
with open(output_file, 'w') as file: # Open the file in write mode

    for i in range(len(run_list)):
        # Write the "s s i" hdtv command
        file.write(f"s s {i}\n")

        # Write the command for writing the fits
        file.write(f"fit read {fitspath}/cebraE{detnum}_run_{run_list[i]} \n")

11. This allows you to then read back in the fits you made if you close hdtv. Just repeat step 3 to read in the spectra and then execute the 'fits_read_det#.txt' file to load back in those fits. 

12. Let's load these xml files into python. It's a bit tricky, so use the function below.

In [1]:
def general_xml(file):
    '''
    This function is the most up-to-date, general function that takes in the passed xml file and simply writes a calibrated & uncalibrated
    data file outputted at the same directory location where the code is called from.
    '''
    mytree = ET.parse(file)
    myroot = mytree.getroot()
 
    uncal_fit_list = []
    uncal_fit_err_list = []
    uncal_width_list = []
    uncal_width_err_list = []
    uncal_volume_list = []
    uncal_volume_err_list = []

    cal_fit_list = []
    cal_fit_err_list = []
    cal_width_list = []
    cal_width_err_list = []
    cal_volume_list = []
    cal_volume_err_list = []

    for fit in myroot:
        for i in fit:
            if i.tag == 'peak':
                for child in i.iter():
                    if child.tag == 'uncal':
                        for j in child.iter():
                            if j.tag == 'pos':
                                for newchild in j.iter():
                                    if newchild.tag == 'value':
                                        fit_value = newchild.text
                                        uncal_fit_list.append(float(fit_value))
                                    elif newchild.tag == 'error':
                                        fit_err = newchild.text
                                        uncal_fit_err_list.append(float(fit_err))
                            elif j.tag == 'vol':
                                for newchild in j.iter():
                                    if newchild.tag == 'value':
                                        vol_value = newchild.text
                                        uncal_volume_list.append(float(vol_value))
                                    elif newchild.tag == 'error':
                                        vol_err = newchild.text
                                        uncal_volume_err_list.append(float(vol_err))
                            elif j.tag == 'width':
                                for newchild in j.iter():
                                    if newchild.tag == 'value':
                                        width_value = newchild.text
                                        uncal_width_list.append(float(width_value))
                                    elif newchild.tag == 'error':
                                        width_err = newchild.text
                                        uncal_width_err_list.append(float(width_err))

                    #gets the calibrated data information                
                    if child.tag == 'cal':
                        for j in child.iter():
                            if j.tag == 'pos':
                                for newchild in j.iter():
                                    if newchild.tag == 'value':
                                        fit_value = newchild.text
                                        cal_fit_list.append(float(fit_value))
                                    elif newchild.tag == 'error':
                                        fit_err = newchild.text
                                        cal_fit_err_list.append(float(fit_err))
                            elif j.tag == 'vol':
                                for newchild in j.iter():
                                    if newchild.tag == 'value':
                                        vol_value = newchild.text
                                        cal_volume_list.append(float(vol_value))
                                    elif newchild.tag == 'error':
                                        vol_err = newchild.text
                                        cal_volume_err_list.append(float(vol_err))
                            elif j.tag == 'width':
                                for newchild in j.iter():
                                    if newchild.tag == 'value':
                                        width_value = newchild.text
                                        cal_width_list.append(float(width_value))
                                    elif newchild.tag == 'error':
                                        width_err = newchild.text
                                        cal_width_err_list.append(float(width_err))
    
    # uncalibrated data handling
    uncal_list = []
    for val in zip(uncal_fit_list, uncal_fit_err_list, uncal_width_list, uncal_width_err_list, uncal_volume_list, uncal_volume_err_list):  #interleaves lists together
        uncal_list.append(val)
    sorted_uncal_list = sorted(uncal_list, reverse=True)

    # calibrated data handling

    cal_list = []
    for val in zip(cal_fit_list, cal_fit_err_list, cal_width_list, cal_width_err_list, cal_volume_list, cal_volume_err_list):  #interleaves lists together
        cal_list.append(val)
    sorted_cal_list = sorted(cal_list, reverse=True)

    return cal_list

In [ ]:
from glob import glob
import numpy as np
# Loading in xml files for each detector

# First get the directories 
det0dir = '/home/dhoulihan/Projects/SPS_CEBRA_Oct2022ad/Analysis/CeBrA_cal/fits/det_0'
det1dir = '/home/dhoulihan/Projects/SPS_CEBRA_Oct2022ad/Analysis/CeBrA_cal/fits/det_1'
det2dir = '/home/dhoulihan/Projects/SPS_CEBRA_Oct2022ad/Analysis/CeBrA_cal/fits/det_2'
det3dir = '/home/dhoulihan/Projects/SPS_CEBRA_Oct2022ad/Analysis/CeBrA_cal/fits/det_3'
det4dir = '/home/dhoulihan/Projects/SPS_CEBRA_Oct2022ad/Analysis/CeBrA_cal/fits/det_4'

# Sorting the files in the directory specified above
det_0_xml = sorted(glob(f'{det0dir}/*.fit'))
det_1_xml = sorted(glob(f'{det1dir}/*.fit'))
det_2_xml = sorted(glob(f'{det2dir}/*.fit'))
det_3_xml = sorted(glob(f'{det3dir}/*.fit'))
det_4_xml = sorted(glob(f'{det4dir}/*.fit'))

# converting from xml to txt files so I can work with them in python
det0 = [];det1 = [];det2 = [];det3 = [];det4 = []
for i in range(len(det_0_xml)):
    det0.append(general_xml(det_0_xml[i]))
    det1.append(general_xml(det_1_xml[i]))
    det2.append(general_xml(det_2_xml[i]))
    det3.append(general_xml(det_3_xml[i]))
    det4.append(general_xml(det_4_xml[i]))

13. The next step is to choose a standard run that you would shift all other runs to. Pick a run that has good statistics and clear peaks.

14. Now we want to shift all of the other spectra's peaks to this set of standard peaks. The way I did it was to loop through them individually and plot the standard peak positions vs. the individual spectrum peak positions. Then you fit a line to it and apply those parameters to the spectrum so that the channels are shifted correctly. I used opt.curve_fit for this, but you can use any other fitting package or technique you need. There is an example code below:

In [ ]:
import scipy.optimize as opt
# Picking which spectrum will be used as the standard to which everything will be shifted
standard = np.array(det0[28]) 
# Extracting the peak positions 
stpeaks = standard[:,0]

# The function that opt.curve_fit will be using, I use a second-order polynomial
def func(x,a,b,c):
    return a*x**2 + b*x + c

# Defining what detector I am going to fit to the standard run
det = det0
runs = len(det)

# Initializing arrays for the loop
fit_params = np.zeros((runs,3))
dfit_params = np.zeros((runs,3))

diff = np.zeros((runs,8)); Ddiff = np.zeros((runs,8)) # the 8 refers to the amount of peaks I fit, so change this to how many you did
peaks = np.zeros((len(det),8)); dpeaks = np.zeros((len(det),8))
pcal = np.zeros((len(det),8)) ; dpcal = np.zeros((len(det),8))

# for looping through all the runs for one detector
# You would have to do this for each detector
for i in range(len(det)):

    peaks[i] = np.array(det[i])[:,0]
    dpeaks[i] = np.array(det[i])[:,1]

    p0 = [(0,1,0)] # an initial guess for the opt.curve_fit function
    popt, pcov = opt.curve_fit(func, peaks[i], stpeaks, p0 = p0)
    perr= np.sqrt(np.diag(pcov))
   
    # I save each set of parameters and their uncertainties in these arrays
    fit_params[i,:] = popt
    dfit_params[i,:] = perr

    # This is looping through all the fitted peaks, so change the range depending on how many peaks you fit
    for j in range(8):

        pcal[i,j] = func(peaks[i], *fit_params[i])[j]
        # I also calculate the difference between the fitted and standard peaks for residual plots later
        diff[i,j] = stpeaks[j] - pcal[i,j]

        dpcal[i,j] = np.sqrt(((2*fit_params[i,0]+fit_params[i,1])*dpeaks[i,j])**2+(peaks[i,j]**2*dfit_params[i,0])**2+(peaks[i,j]*dfit_params[i,1])**2+(dfit_params[i,2])**2)
        

        Ddiff[i,j] = np.sqrt((standard[j,1])**2+(dpcal[i,j])**2)

# You could plot some figures as a sanity check and make sure the fitting is going well
# Fitting more peaks will make the fit better


You could also use this function to get the fit parameters (it's the same procedure). See below:

In [ ]:
def fitruns(detlist, standardrun, detnum, out):

    standard = np.array(standardrun)
    stpeaks = standard[:,0]

    p0 = [(1,1,0)]
    fit_params = np.zeros((len(detlist),3))
    dfit_params = np.zeros((len(detlist),3))

    for i in range(len(detlist)):

        popt, pcov = opt.curve_fit(func, np.array(detlist[i])[:,0], stpeaks, p0 = p0)
        perr= np.sqrt(np.diag(pcov))
        #data_fitted = func(np.array(det0[i])[:,0], *popt)
        fit_params[i,:] = popt
        dfit_params[i,:] = perr

    if out == True:
        output(detnum,fit_params)
    return fit_params, dfit_params

# function takes peaks and fits a line to get fit parameters 
det0fits = fitruns(det0,det0[28],0,True)
det1fits = fitruns(det1,det0[28],1,True)
det2fits = fitruns(det2,det0[28],2,True)
det3fits = fitruns(det3,det0[28],3,True)
det4fits = fitruns(det4,det0[28],4,True)

detfits = []
detfits.append(det0fits)
detfits.append(det1fits)
detfits.append(det2fits)
detfits.append(det3fits)
detfits.append(det4fits)

15. It's important to check our work and make sure things make sense. That means making some plots! One such plot is a residual plot. Basically, I plotted the difference between the calibrated peaks and the standard peak positions to see if there are any big discrepancies. Use the code below to do so for one detector:

In [3]:
import matplotlib.pyplot as plt
# Just making an array of x values corresponding to each peak I fit. In this case, there were 8 peaks.
x = np.linspace(0,7, num = 8)

# Specifies which detector I'm looking at
detnum = 3

plt.figure()
# Making a vertical line at 0 - ideally the difference should be zero
plt.axhline(y = 0, color= 'black')
for i in range(len(det)):
    # plt.errorbar(x, diff[i,:], yerr = Ddiff[i,:],marker = 'o', capsize = 3, linestyle = 'None')
    plt.scatter(x,diff[i,:])

plt.xlabel('Peak Number')
plt.ylabel('Residuals from standard peaks (pos)')
plt.title(f'Detector {detnum}')
plt.show()

Tip: Remember that the CeBrA spectra goes from 0 to 4096, with a bin width of 8 channels. So it might be a good idea to divide the difference by 8 and see how far off the peaks are in terms of channel bins. 

16. Now, we are going to have to deal with the actual ROOT data in python, which can be tricky because the files are big. So these next two code blocks will do: (a) Create new ROOT Trees with the Gain-matched CeBrA data and put them into separate ROOT files and (b) Take those new gain-matched files and do PID cuts on them so that they are easier to work with (much smaller data sets). 

Tip: Make sure to put these files in different folders and make sure not to overwrite any of the original analyzed files.

In [ ]:
def histo1d(Frame, Name, X, X_List):
   #X_List should be something like X_List = [bins,initial,final]
   histo1d = Frame.Histo1D((f"{Name}",f"{Name}",X_List[0], X_List[1], X_List[2]),X)
   return histo1d


path = '/home/dhoulihan/Projects/SPS_CEBRA_Oct2022ad/Workingdir/analyzed/'

#Creates a list of run numbers give it the path.  Directory should only have the trees that you want to use
run_list = []
for filename in os.listdir(path):
   if filename.endswith(".root"):
       run_number = int(filename.split("_")[1].split(".")[0])
       run_list.append(run_number)

run_list.sort()


column_names = {"anodeBack", "anodeBackTime", "anodeFront", "anodeFrontTime", "cathode", "cathodeTime",\
                "cebraE0", "cebraE1", "cebraE2", "cebraE3", "cebraE4", "cebraTime0", "cebraTime1", "cebraTime2",\
                 "cebraTime3", "cebraTime4", "delayBackLeftE", "delayBackLeftTime", \
                 "delayBackRightE",  "delayBackRightTime", "delayFrontLeftE", \
                 "delayFrontLeftTime", "delayFrontRightE", "delayFrontRightTime", \
                 "scintLeft", "scintLeftTime", "scintRight", "scintRightTime", "theta", "x1", "x2", "xavg"\
                , "cebraE0_GM", "cebraE1_GM", "cebraE2_GM", "cebraE3_GM", "cebraE4_GM", \
                'cebraTime0_toScint', 'cebraTime1_toScint', 'cebraTime2_toScint', 'cebraTime3_toScint', 'cebraTime4_toScint' }

for i in range(len(run_list)):
   

    print(f"Run {run_list[i]}")

    df = ROOT.RDataFrame("SPSTree",f"{path}run_{run_list[i]}.root")
    cebraE_raw_to_ECal = []

    for j in range(5):
        
        # df_i = df.Filter(f"cebraE{j} != -1")
        # PID_filter_cond = f"anodeBackTime != -1e6 && xavg != -1e6 && scintLeft > {SLmin} && scintLeft < {SLmax} && anodeBack > {ABmin} && anodeBack < {ABmax}"
        # root_df_filtered = df_i.Filter(f"{PID_filter_cond}") # applying PID filter

        a = detfits[j][0][i,0]
        b = detfits[j][0][i,1]
        c = detfits[j][0][i,2]

        caldata = f"{a}*cebraE{j}*cebraE{j} + {b}*cebraE{j} + {c}"
        df = df.Define(f'cebraE{j}_GM', caldata)
        
        df = df.Define(f'cebraTime{j}_toScint', f'cebraTime{j} - scintLeftTime')


    df.Snapshot('SPSTree', f"{path}/GainMatched/run_{run_list[i]}.root", column_names)

# Uncomment this section when you are ready to save to a new ROOT file

#    output = ROOT.TFile.Open(f"run_{run_list[i]}_cebraEnergyGM.root", "RECREATE")
#    output.cd()
#    for k in range(5):
#        output.mkdir(f"det{k}_GM")
#        output.cd(f"det{k}_GM")
#        cebraE_raw_to_ECal[k].Write()
   
#    output.Close()

In [ ]:
GMpath = '/home/dhoulihan/Projects/SPS_CEBRA_Oct2022ad/Workingdir/analyzed/GainMatched/'
path = '/home/dhoulihan/Projects/SPS_CEBRA_Oct2022ad/Workingdir/analyzed/'

#Creates a list of run numbers give it the path.  Directory should only have the trees that you want to use
run_list = []
for filename in os.listdir(GMpath):
   if filename.endswith(".root"):
       run_number = int(filename.split("_")[1].split(".")[0])
       run_list.append(run_number)

run_list.sort()

# print(len(run_list))

column_names = {"anodeBack", "anodeBackTime", "anodeFront", "anodeFrontTime", "cathode", "cathodeTime",\
                "cebraE0", "cebraE1", "cebraE2", "cebraE3", "cebraE4", "cebraTime0", "cebraTime1", "cebraTime2",\
                 "cebraTime3", "cebraTime4", "delayBackLeftE", "delayBackLeftTime", \
                 "delayBackRightE",  "delayBackRightTime", "delayFrontLeftE", \
                 "delayFrontLeftTime", "delayFrontRightE", "delayFrontRightTime", \
                 "scintLeft", "scintLeftTime", "scintRight", "scintRightTime", "theta", "x1", "x2", "xavg"\
                , "cebraE0_GM", "cebraE1_GM", "cebraE2_GM", "cebraE3_GM", "cebraE4_GM", \
                'cebraTime0_toScint', 'cebraTime1_toScint', 'cebraTime2_toScint', 'cebraTime3_toScint', 'cebraTime4_toScint' }

start = 113; finish = 134
SLABpar = [[335,750,470,1290],[189,994,559,1290],[118,395,685,1402],\
           [68, 311,800,1460],[61, 230,920,1520],[12,152,1145,1640]]

j = 5
SLmin = SLABpar[j][0]; 
SLmax = SLABpar[j][1]; 
ABmin = SLABpar[j][2]; 
ABmax = SLABpar[j][3]

for i in run_list:
        if i >= start:
                if i <= finish:
    

                    print(f"Run {i}")

                    df = ROOT.RDataFrame("SPSTree",f"{GMpath}run_{i}.root")
                   

                    PID_filter_cond = f"anodeBackTime != -1e6 && xavg != -1e6 && scintLeft > {SLmin} && scintLeft < {SLmax} && anodeBack > {ABmin} && anodeBack < {ABmax}"
                    root_df_filtered = df.Filter(f"{PID_filter_cond}") # applying PID Cut

                    
                    root_df_filtered.Snapshot('SPSTree', f"{path}GMandcut/run_{i}.root", column_names)

17. Now that the CeBrA detectors are gain-matched, let's try and combine them into a summed histogram. First, we need to load in the data. We can use the function below to get the CeBrA data from the gain-matched and cut ROOT files. The inputs include 'start' and 'finish' which refere to the run numbers of each set of runs (same magnetic field settings). You will also have to look at the timing peaks in the histograms produced from the EventBuilder to determine the peak position (tshift) and its width (twidth). This ensures that the data is now time-cut, which means only gamma rays that were detected IN COINCIDENCE with a measured ejectile is shown. If you want ALL gamma data, edit the function to remove the time cuts! Gamma is a True/False input asking whether you want gamma data or xavg data. In this case, we will put True.

In [ ]:

def combineCEandxavg(start, finish, path, tshift,twidth,gamma):

    run_list = []
    for filename in os.listdir(path):
        if filename.endswith(".root"):
            run_number = int(filename.split("_")[1].split(".")[0])
            run_list.append(run_number)

    run_list.sort()

    
    combine_detCE = []; combine_detxavg = []


    for i in run_list:
        if i >= start:
            if i <= finish:

                # print(i)
                df = ROOT.RDataFrame("SPSTree",f"{path}run_{i}.root")
                    
                final_CE = []; finalCEarr = []
                final_xavg = []; finalxavgarr = []                 
            
                for j in range(5):

                    df_i = df.Filter(f"cebraE{j} != -1")
                    

                    dettimecut1 = df_i.Define(f"cebra_RelTime_toScint_{j}", f"cebraTime{j} - scintLeftTime + {tshift[j]}")
                    timecut = f"cebra_RelTime_toScint_{j} > {-twidth} && cebra_RelTime_toScint_{j} < {twidth}"
                    dettimecut = dettimecut1.Filter(f"{timecut}")

                    final_CE.append(pd.DataFrame(dettimecut.AsNumpy(columns = [f'cebraE{j}_GM'])))
                    finalCEarr.append(np.array(final_CE[j][f'cebraE{j}_GM']))

                    final_xavg.append(pd.DataFrame(dettimecut.AsNumpy(columns = ['xavg'])))
                   
                
                combine_detCE.append(np.concatenate(finalCEarr, axis = 0))
                combine_detxavg.append(pd.concat(final_xavg, axis = 0))

    totaldetCE = np.concatenate(combine_detCE, axis = 0)
    totaldetxavg = pd.concat(combine_detxavg, axis = 0)
    
    if gamma == True:
        return totaldetCE
    if gamma == False:
        return totaldetxavg

In [ ]:
# Set your path to gain-matched and cut data
path = '/home/dhoulihan/Projects/SPS_CEBRA_Oct2022ad/Workingdir/analyzed/GMandcut/'
# List of timing peak positions for each detector in each set of runs
tshift = [[710,708,707,707,677], [722,721,720,719,690],[736,733,734,733,703],\
          [746,745,745,744,714],[757,756,754,753,724],[774,772,772,771,742]]
twidth = 6
# ScintLeft and AnodeBack positions that constitute the PID cut
SLABpar = [[335,750,470,1290],[189,994,559,1290],[118,395,685,1402],\
           [68, 311,800,1460],[61, 230,920,1520],[12,152,1145,1640]]

In [ ]:
# getting cebra det info
runs4_30 =  combineCEandxavg(4,30,path, tshift[0],  twidth, True)
runs31_38 = combineCEandxavg(31,38,path, tshift[1], twidth, True)
runs39_53 = combineCEandxavg(39,53,path, tshift[2], 8, True)
runs68_84 = combineCEandxavg(68,84,path, tshift[3], 10, True)
runs85_112 =combineCEandxavg(85,112,path, tshift[4], 10, True)
runs113_134=combineCEandxavg(113,134,path, tshift[5],10, True)

# append it altogether 
allruns = []
allruns.append(runs4_30)
allruns.append(runs31_38)
allruns.append(runs39_53)
allruns.append(runs68_84)
allruns.append(runs85_112)
allruns.append(runs113_134)

18. Once you're done running the function, we can combine all the CeBrA data together and then plot a histogram. Congratulations, you now have a combined coincident gamma-ray spectrum from CeBrA!

In [ ]:
# append it altogether 
allruns = []
allruns.append(runs4_30)
allruns.append(runs31_38)
allruns.append(runs39_53)
allruns.append(runs68_84)
allruns.append(runs85_112)
allruns.append(runs113_134)

# putting this all into an array
totalgrays = np.concatenate(allruns, axis = 0)

plt.figure()
plt.hist(totalgrays, bins = 512, range = [0,4096], histtype= 'step', color = 'black')
plt.xlabel('channels')
plt.ylabel('Counts')
plt.title('Summed Coincident $\gamma$ rays for all runs')


## CeBrA Energy Calibration

There are some steps needed before beginning this procedure. In order to energy calibration the CeBrA detectors, one has to gate on specific excited states to then fit peaks and so on. So in order to energy calibrate CeBrA, we need to make an uncalibrated particle-$\gamma$ matrix. The previous sections must be completed, meaning CeBrA is gain-matched and combined, and the xavg data is energy-calibrated and normalized correctly. The particle-$\gamma$ matrix contains histograms from coincident data only, so time cuts will need to be made to both the CeBrA and particle spectra. Let's get started:

1. After completing the SPS focal plane energy calibration and normalization, we can proceed to make time cuts so that only the particles that have a coincident $\gamma$ ray are plotted. The function below will give you the time-cut xavg data for a set of runs, it just needs the correct inputs. The inputs include 'start' and 'finish' which refer to the run numbers of each set of runs (same magnetic field settings). You will also have to look at the timing peaks in the histograms produced from the EventBuilder to determine the peak position (tshift) and its width (twidth). This ensures that the data is now time-cut, which means only particles that were detected IN COINCIDENCE with a measured $\gamma$ ray is shown. If you want ALL gamma data, edit the function to remove the time cuts! Gamma is a True/False input asking whether you want gamma data or xavg data. In this case, we will put False.

In [ ]:
import pandas as pd
def combineCEandxavg(start, finish, path, tshift,twidth,gamma):

    run_list = []
    for filename in os.listdir(path):
        if filename.endswith(".root"):
            run_number = int(filename.split("_")[1].split(".")[0])
            run_list.append(run_number)

    run_list.sort()

    
    combine_detCE = []; combine_detxavg = []


    for i in run_list:
        if i >= start:
            if i <= finish:

                # print(i)
                df = ROOT.RDataFrame("SPSTree",f"{path}run_{i}.root")
                    
                final_CE = []; finalCEarr = []
                final_xavg = []; finalxavgarr = []                 
            
                for j in range(5):

                    df_i = df.Filter(f"cebraE{j} != -1")
                    

                    dettimecut1 = df_i.Define(f"cebra_RelTime_toScint_{j}", f"cebraTime{j} - scintLeftTime + {tshift[j]}")
                    timecut = f"cebra_RelTime_toScint_{j} > {-twidth} && cebra_RelTime_toScint_{j} < {twidth}"
                    dettimecut = dettimecut1.Filter(f"{timecut}")

                    final_CE.append(pd.DataFrame(dettimecut.AsNumpy(columns = [f'cebraE{j}_GM'])))
                    finalCEarr.append(np.array(final_CE[j][f'cebraE{j}_GM']))

                    final_xavg.append(pd.DataFrame(dettimecut.AsNumpy(columns = ['xavg'])))
                   
                
                combine_detCE.append(np.concatenate(finalCEarr, axis = 0))
                combine_detxavg.append(pd.concat(final_xavg, axis = 0))

    totaldetCE = np.concatenate(combine_detCE, axis = 0)
    totaldetxavg = pd.concat(combine_detxavg, axis = 0)
    
    if gamma == True:
        return totaldetCE
    if gamma == False:
        return totaldetxavg

Here is an example of me using it for another data set:

In [ ]:
# Defining inputs 
path = '/home/dhoulihan/Projects/SPS_CEBRA_Oct2022ad/Workingdir/analyzed/GMandcut/'
tshift = [[710,708,707,707,677], [722,721,720,719,690],[736,733,734,733,703],\
          [746,745,745,744,714],[757,756,754,753,724],[774,772,772,771,742]]
twidth = 6
SLABpar = [[335,750,470,1290],[189,994,559,1290],[118,395,685,1402],\
           [68, 311,800,1460],[61, 230,920,1520],[12,152,1145,1640]]

# Running the function for each set of runs
xavg4_30 =  combineCEandxavg(4,30,path, tshift[0],twidth,   False)
xavg31_38 = combineCEandxavg(31,38,path, tshift[1],twidth,  False)
xavg39_53 = combineCEandxavg(39,53,path, tshift[2],8,  False)
xavg68_84 = combineCEandxavg(68,84,path, tshift[3],10,  False)
xavg85_112 =combineCEandxavg(85,112,path, tshift[4],10, False)
xavg113_134=combineCEandxavg(113,134,path, tshift[5],10,False)

2. We need to then energy calibrate this data. Below shows how you can do that with the parameters you found in the first section.

In [ ]:
# Defining a second order polynomial to convert the data to energy
def Ecal(x,a,b,c):
    return a*x**2 + b*x + c

# Defining the fit parameters determined earlier for each set of runs
fitpar = [[-0.0030797,-14.3656, 1984.7056],[-0.00204,-11.03256,5251.766]\
          ,[-0.0008726,-8.6791,7840.8340],[0.00329886,-8.18386,9405.789],[0.0031105,-8.31563,10585.9494]\
          ,[0.002247, -8.28524,12746.2791]]

# Converting each set of runs into energy 
xavgcal4_30   = Ecal(xavg4_30['xavg'], *fitpar[0])
xavgcal31_38  = Ecal(xavg31_38['xavg']  , *fitpar[1])
xavgcal39_53  = Ecal(xavg39_53['xavg']  , *fitpar[2])
xavgcal68_84  = Ecal(xavg68_84['xavg']  , *fitpar[3])
xavgcal85_112 = Ecal(xavg85_112['xavg'] , *fitpar[4])
xavgcal113_134= Ecal(xavg113_134['xavg'], *fitpar[5])

3. Combine all the data into one array.

In [ ]:
totaldeutE = np.concatenate((xavgcal4_30, xavgcal31_38, xavgcal39_53, xavgcal68_84, xavgcal85_112, xavgcal113_134))

4. Make an array of the scaling factors for the xavg normalization to be used for the combined xavg data. Again, an example from another data set is shown below.

In [ ]:
# list of the scaling factors
s = [1,2.02,1.68,1.14,0.89,0.93]
# Making one big list for the scaling factors to be put in according to what run set they're apart of
sfactors = []
sfactors.append(np.full(len(xavgcal4_30),s[0]))
sfactors.append(np.full(len(xavgcal31_38),s[1]))
sfactors.append(np.full(len(xavgcal39_53),s[2]))
sfactors.append(np.full(len(xavgcal68_84),s[3]))
sfactors.append(np.full(len(xavgcal85_112),s[4]))
sfactors.append(np.full(len(xavgcal113_134),s[5]))

# combining them into an array
totweights = np.concatenate(sfactors)

5. Do a similar procedure for CeBrA. Use the same CombineCEandxavg function, but put gamma to True.

In [ ]:
# getting cebra det info
runs4_30 =  combineCEandxavg(4,30,path, tshift[0],  twidth, True)
runs31_38 = combineCEandxavg(31,38,path, tshift[1], twidth, True)
runs39_53 = combineCEandxavg(39,53,path, tshift[2], 8, True)
runs68_84 = combineCEandxavg(68,84,path, tshift[3], 10, True)
runs85_112 =combineCEandxavg(85,112,path, tshift[4], 10, True)
runs113_134=combineCEandxavg(113,134,path, tshift[5],10, True)

6. Combine the run sets into an array.

In [ ]:
allruns = []
allruns.append(runs4_30)
allruns.append(runs31_38)
allruns.append(runs39_53)
allruns.append(runs68_84)
allruns.append(runs85_112)
allruns.append(runs113_134)

totalgrays = np.concatenate(allruns, axis = 0)

7. Now we can plot these histograms into a matrix. Make sure the binning and ranges are correct for the energy-calibrated xavg spectrum.

In [ ]:
import matplotlib.colors as colors
# energy range of xavg
xmin = -2602; xmax = 15434
# xavg bins
xbins = 1202

# The range of CeBrA channels
ymin = 0; ymax = 4096
# CeBrA bins
Cbins = 512

plt.figure()
plt.hist2d(totaldeutE, totalgrays, bins = [xbins,Cbins], range = [[xmin,xmax],[ymin,ymax]], weights = np.full(len(totaldeutE),totweights), \
           cmap = 'viridis', norm = colors.LogNorm(), alpha = 0.8)
plt.ylabel('CeBrA Channels', fontsize = 13)
plt.xlabel('$^{24}$Si Energy (keV)', fontsize = 13)


8. Check and make sure it makes sense. Assuming you have enough counts, you should see horizontal bands corresponding to different gamma decays and diagonals corresponding to different full-energy decays.

9. Once you are confident it looks good, let's save it to a ROOT file and do the rest of the analysis in TBrowser:

In [ ]:
output = ROOT.TFile.Open(f"pgmatrix_uncal.root", "RECREATE") # you can call it what you want here
output.cd()

hist = ROOT.TH2F('deutgammamatrix', 'deutgammamatrix', 1202, -2602, 15434, 512, 0, 4096) # make sure the bin numbers and ranges are correct
for i in range(len(totaldeutE)):
    hist.Fill(totaldeutE[i],totalgrays[i])
    
hist.Write()
output.Close()

10. Open the matrix in TBrowser by typing 'root filename.root' and then typing 'new TBrowser'. 

11. Open the matrix, right click on it, and click 'SetShowProjectionY'.

12. A small window will pop up, type a number in (it is the number of bins so choose as big or small as you find fit) and press enter.

13. Another spectrum will pop up, showing the yprojection of whatever x window you choose. Hover over an excited state (a lower-lying one to start) and carefully move the cursor out from the matrix without having the yprojection move away from that state. 

14. Go to the pop-up window and click tools -> Fit Panel.

15. On the Fit Panel, make sure it says 'gaus' under operation.

16. Use the sliding scale on the bottom to create a range for your fit.

17. Click 'fit' to fit a peak that you want.

18. Look at the terminal and record the 'Mean' data in a spreadsheet. 

19. Look on nndc and cross-reference gamma-decay energies with your peaks and try to fit a second-order polynomial. It's the same procedure as the xavg energy calibration.

20. Gate on several excited states and fit more peaks. Lower lying decays will, or should, show up in higher-lying states. So you can take an average of the positions for a specific gamma decay and use that in your fit.

21. Find fit parameters and use that to calibrate the CeBrA channels to energy. This is done by using the Ecal function on the totalgrays array (or your equivalent array). 

22. Repeat steps 8-9 and check again that the energies of the new calibrated spectrum match well with what you expect. 

### Congratulations, you now have an energy-calibrated particle-$\gamma$ matrix that you can now do some scientific analysis on! Good Luck!